In [ ]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df1 = pd.read_csv('sharegptTrain.csv', index_col=0)
df1.columns = ['text', 'lang']
df1['labels'] = 1
df2 = pd.read_csv('negative.csv', index_col=0)
df2 = df2[df2.text.notnull()]
df2['labels'] = 0

In [ ]:
df1 = df1.drop('lang', axis=1)
df3 = df1.append(df2, ignore_index=True)

<ipython-input-3-e59d4e15137d>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df1.append(df2, ignore_index=True)


In [ ]:
from sklearn.utils import shuffle
df = shuffle(df3)
df

,text,labels
94117,"it's also why any aliens if they exist, they h...",0
10104,hiHello! How may I assist you today?如果没有孙中山，中国...,1
122456,"if ur pasta is laced with opioids, then sure.....",0
29710,Please generate code in SAS for the following ...,1
47085,describe a specific rube goldburg in detail,1
...,...,...
48109,あああああ,1
101205,bye,0
62724,O jaki rodzaj wypowiedzi pisemnej prosi nauczy...,1
99214,The medium is also the message.,0


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

In [ ]:
tokenized_imdb = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/124398 [00:00<?, ? examples/s]

Map:   0%|          | 0/13823 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.5 MB/s eta 0:00:00


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels), 

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.111900,0.109645,0.965565
2,0.060800,0.111862,0.969761


TrainOutput(global_step=15550, training_loss=0.10293321959075437, metrics={'train_runtime': 11576.5731, 'train_samples_per_second': 21.491, 'train_steps_per_second': 1.343, 'total_flos': 3.2957358916018176e+16, 'train_loss': 0.10293321959075437, 'epoch': 2.0})

In [ ]:
stuff=trainer.predict(tokenized_imdb["test"])


In [ ]:
stuff.predictions.argmax(axis=1)

array([0, 0, 1, ..., 1, 1, 0])

In [ ]:
from sklearn.metrics import classification_report
y_true = tokenized_imdb["test"]['labels']
y_pred = stuff.predictions.argmax(axis=1)
target_names = ['0', '1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      5101
           1       0.98      0.97      0.97      8722

    accuracy                           0.97     13823
   macro avg       0.96      0.96      0.96     13823
weighted avg       0.97      0.97      0.97     13823



In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file runs/Apr07_22-30-23_37be4a9c93c4/events.out.tfevents.1680906692.37be4a9c93c4.531.0:   0%|         …

To https://huggingface.co/1112lee/my_awesome_model
   e19fa56..a6e9099  main -> main

   e19fa56..a6e9099  main -> main

To https://huggingface.co/1112lee/my_awesome_model
   a6e9099..ec81ca9  main -> main

   a6e9099..ec81ca9  main -> main



'https://huggingface.co/1112lee/my_awesome_model/commit/a6e9099f1d0a35bee8ae84ae783fc60c128e4b58'

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
!pip install googletrans

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
df = pd.read_csv('biggpt.csv')

In [ ]:
df.loc[df['0'].str.contains('Question Requested \n >'), '0']= df[df['0'].str.contains('Question Requested \n >')]['0'].str.split('Question Requested \n > ').str[1].str.split('\n```').str[0]

In [ ]:
df.loc[df['0'].str.contains('<@1055196262896500776>'), '0']= df[df['0'].str.contains('<@1055196262896500776>')]['0'].str.split('<@1055196262896500776>').str[1]

In [ ]:
df.loc[df['0'].str.contains('<@1064152790181609532>'), '0']= df[df['0'].str.contains('<@1064152790181609532>')]['0'].str.split('<@1064152790181609532>').str[1]

In [ ]:
df.loc[df['0'].str.contains('\*\*'), '0']= df[df['0'].str.contains(r'\*\*')]['0'].str.split('\*\*').str[2]

In [ ]:
df = df[df['0'].notnull()]

In [ ]:
df[df['0'].str.contains(r'\*\*')]['0'].str.split('\*\*').str[2]

Series([], Name: 0, dtype: object)

In [ ]:
df['lang'] = 'Ch'

In [ ]:
df['lang']

0         Ch
1         Ch
2         Ch
3         Ch
4         Ch
          ..
143770    Ch
143771    Ch
143772    Ch
143773    Ch
143774    Ch
Name: lang, Length: 143731, dtype: object

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
translist = list()

In [ ]:
from langdetect import detect
def translate(row):
  try:
    mess.append(translator.translate(row['0'], dest='en'))
  except:
    mess.append('no')

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df.progress_apply(translate, axis=1)

 36%|███▌      | 51991/143731 [2:20:20<5:28:41,  4.65it/s]

In [ ]:
 df['lang'] = translist

In [ ]:
df[df.lang == 'es']

,Unnamed: 0,0,lang
407,407,define anajabactor,es
420,420,Resumen de Ciencia y de los conceptos: Princi...,es
422,422,Resumen de Ciencia y de los conceptos: Princi...,es
423,423,crea un cuadro sinoptico donde la palabra pri...,es
472,472,Escribe un ejemplo de literatura erótica deta...,es
...,...,...,...
143243,143243,cual es la criptomoneda mas estable,es
143343,143343,un poema para mi esposa,es
143357,143357,Valentines day ideas,es
143458,143458,centos7 安装docker,es


The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for sentiment analysis with your model, and pass your text to it:

In [ ]:
stuff = list()

In [ ]:
from transformers import pipeline

classifier = pipeline(model="1112lee/my_awesome_model")

In [ ]:
def ai(row):
  try: 
    stuff.append(int(classifier(str(row['0']))[0]['label'][-1]))
  except:
    print(row)

In [ ]:
df.apply(ai, axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


Unnamed: 0                                                  475
0             **Kibou未来#8735:** \nX = group_dataset.drop(col...
Name: 475, dtype: object
Unnamed: 0                                                  502
0              # Clear the contents of ip.txt before the loo...
Name: 502, dtype: object
Unnamed: 0                                                  513
0             **Iris2hq#9087:** fix syntax errors\n\nwhile T...
Name: 513, dtype: object
Unnamed: 0                                                  514
0              fix syntax errors\n\nwhile True:\n    # Use d...
Name: 514, dtype: object
Unnamed: 0                                                  520
0               modify my code to use two driver instances t...
Name: 520, dtype: object
Unnamed: 0                                                  523
0               I want my code to use two tabs to complete t...
Name: 523, dtype: object
Unnamed: 0                                                  552
0             **Ла